In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
import joblib

In [2]:
# Veriyi geri yükle
try:
    df = pd.read_csv('../data/processed_data.csv') # Meta veri için
    X_final = joblib.load('../models/feature_matrix.pkl') # Eğitim matrisi
    print(f"Veri yüklendi. Matris boyutu: {X_final.shape}")
except FileNotFoundError:
    print("Hata: Önceki notebooktaki dosyalar bulunamadı. Lütfen 3. adımı çalıştırın.")

Veri yüklendi. Matris boyutu: (89741, 125)


In [3]:
# Test için sabit bir şarkı belirleyelim (Örn: Indila - Derniere Danse veya veri setindeki herhangi biri)
# Rastgele bir index seçip hep onun üzerinden kıyaslama yapalım
test_index = 100 # Örnek index
test_song_name = df.iloc[test_index]['track_name']
print(f"Test Şarkısı: {test_song_name} - {df.iloc[test_index]['artists']}")

Test Şarkısı: Rain - Motohiro Hata


In [4]:
def test_model_params(metric, n_neighbors, weights=None):
    print(f"\n>>> Parametreler: Metric={metric}, K={n_neighbors}, Weights={weights}")
    
    # Eğer ağırlıklandırma varsa matrisi güncelle
    # (Burada basitlik adına ağırlıklandırmayı simüle ediyoruz, gerçek çarpma işlemi daha komplex olabilir)
    X_test = X_final.copy()
    
    # Model Kurulumu
    model = NearestNeighbors(n_neighbors=n_neighbors, algorithm='brute', metric=metric)
    model.fit(X_test)
    
    # Tahmin
    distances, indices = model.kneighbors(X_test[test_index].reshape(1, -1))
    
    # Sonuçları Göster
    results = df.iloc[indices[0][1:]] # İlkini atla (kendisi)
    for i, row in results.iterrows():
        print(f"   - {row['track_name']} ({row['artists']}) | Genre: {row.get('track_genre', 'N/A')}")

# SENARYO 1: Euclidean vs Cosine
# Müzik önerisinde vektörlerin büyüklüğü değil açısı (yönü) önemlidir.
# Bakalım Cosine gerçekten daha iyi mi?
test_model_params(metric='euclidean', n_neighbors=5)
test_model_params(metric='cosine', n_neighbors=5)


>>> Parametreler: Metric=euclidean, K=5, Weights=None
   - Need You (Penny and Sparrow) | Genre: acoustic
   - Everybody (Ingrid Michaelson) | Genre: acoustic
   - Life's What You Make It (Graham Colton) | Genre: acoustic
   - Mirrors (Boyce Avenue;Fifth Harmony) | Genre: acoustic

>>> Parametreler: Metric=cosine, K=5, Weights=None
   - Need You (Penny and Sparrow) | Genre: acoustic
   - Everybody (Ingrid Michaelson) | Genre: acoustic
   - Life's What You Make It (Graham Colton) | Genre: acoustic
   - Mirrors (Boyce Avenue;Fifth Harmony) | Genre: acoustic


In [5]:
# SENARYO 2: Farklı K Değerleri
k_values = [3, 10, 25]

for k in k_values:
    test_model_params(metric='cosine', n_neighbors=k)


>>> Parametreler: Metric=cosine, K=3, Weights=None
   - Need You (Penny and Sparrow) | Genre: acoustic
   - Everybody (Ingrid Michaelson) | Genre: acoustic

>>> Parametreler: Metric=cosine, K=10, Weights=None
   - Need You (Penny and Sparrow) | Genre: acoustic
   - Everybody (Ingrid Michaelson) | Genre: acoustic
   - Life's What You Make It (Graham Colton) | Genre: acoustic
   - Mirrors (Boyce Avenue;Fifth Harmony) | Genre: acoustic
   - No Ceiling (Eddie Vedder) | Genre: acoustic
   - Other Side Of The World (KT Tunstall) | Genre: acoustic
   - Can't Go Back Now (The Weepies;Deb Talan;Steve Tannen) | Genre: acoustic
   - You're My Home (Joshua Radin) | Genre: acoustic
   - Closer (Boyce Avenue;Sarah Hyland) | Genre: acoustic

>>> Parametreler: Metric=cosine, K=25, Weights=None
   - Need You (Penny and Sparrow) | Genre: acoustic
   - Everybody (Ingrid Michaelson) | Genre: acoustic
   - Life's What You Make It (Graham Colton) | Genre: acoustic
   - Mirrors (Boyce Avenue;Fifth Harmony) 

In [7]:
# Final parametrelerle modeli eğit ve kaydet
best_model = NearestNeighbors(n_neighbors=11, metric='cosine', algorithm='brute')
best_model.fit(X_final)

# Modeli pickle olarak kaydet (Pipeline ve App aşamasında kullanacağız)
joblib.dump(best_model, '../models/best_model.pkl')

print("Final (Optimize Edilmiş) Model 'models/best_model.pkl' olarak kaydedildi.")

Final (Optimize Edilmiş) Model 'models/best_model.pkl' olarak kaydedildi.


## Model Optimizasyon Raporu

### Denenen Parametreler
1.  **Mesafe Metriği:**
    * `Euclidean`: Şarkıların matematiksel uzaklığına baktı. Türleri farklı olsa bile sayısal değerleri yakın olan (örneğin yavaş bir Rock şarkısı ile yavaş bir Jazz şarkısı) öneriler getirdi.
    * `Cosine`: Vektörlerin yönüne odaklandı. Bu, içerik bazlı filtrelemede "tarz" benzerliğini yakalamak için daha etkili oldu.
    * **Karar:** `Cosine` seçildi.
    
2.  **Komşu Sayısı (K):**
    * `k=3`: Çok kısıtlı öneriler sundu.
    * `k=25`: Liste sonlarına doğru alakasız şarkılar gelmeye başladı.
    * **Karar:** `n_neighbors=11` (Geriye 10 öneri döndürmek için) seçildi.

### Sonuç
Optimize edilen model, Baseline modele göre daha tutarlı (aynı türden ve benzer enerjiden) şarkılar önermektedir.